# with updated safegraph data

In [5]:
import pandas as pd
import geopandas as gpd
import json
import requests
import numpy as np
from shapely.geometry import Point

In [6]:
def traveltimeTransit(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,TRANSIT&date=05-04-2022&time="\
    +time+"&maxWalkDistance=750"
    # print(request)
    r = requests.get(request)
    data=r.json()
    # try:
    #     legs = pd.DataFrame(data['plan']['itineraries'][0]['legs'])
    # except:
    #     return np.nan
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [7]:
def traveltimeTransit2(df, i, latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,TRANSIT&date=05-04-2022&time="\
    +time+"&maxWalkDistance=750"
    # print(request)
    r = requests.get(request)
    data=r.json()
    # print('data: ', data)
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        min_dur = data['plan']['itineraries'][0]['duration']
        pos = 0
        # print(data['plan']['itineraries'][0])
        for itin in range(len(data['plan']['itineraries'])):
            if data['plan']['itineraries'][itin]['duration']<min_dur:
                min_dur = data['plan']['itineraries'][itin]['duration']
                pos = itin
        # print('itineriers', int(data['plan']['itineraries'][pos]))
        df.loc[i, 'transit_time'] = int(data['plan']['itineraries'][pos]['duration'])
        df.loc[i, 'walk_percent'] = np.divide(int(data['plan']['itineraries'][pos]['walkTime']), int(data['plan']['itineraries'][pos]['duration']))*100
        df.loc[i, 'waiting_percent'] = np.divide(int(data['plan']['itineraries'][pos]['waitingTime']), int(data['plan']['itineraries'][pos]['duration']))*100
        df.loc[i, 'transfers'] = int(data['plan']['itineraries'][pos]['transfers'])
        legs = pd.DataFrame(data['plan']['itineraries'][pos]['legs'])
        # df.loc[i, 'trips'] = np.array(legs['tripId'].dropna().to_list())
        # df.loc[i, 'trips'] = df.apply(lambda r: tuple(legs['tripId'].dropna()), axis=1).apply(np.array)
        try:
            df.loc[i, 'trip1'] = legs['tripId'].dropna().iloc[0].split(':')[1]
        except:pass
        try:
            df.loc[i, 'trip2'] = legs['tripId'].dropna().iloc[0].split(':')[1]
        except:pass
        try:
            df.loc[i, 'trip3'] = legs['tripId'].dropna().iloc[0].split(':')[2]
        except:pass
        try:    
            df.loc[i, 'route1'] = legs.route[legs.route != ''].iloc[0]
        except:pass
        try:
            df.loc[i, 'route2'] = legs.route[legs.route != ''].iloc[1]
        except:pass
        try:
            df.loc[i, 'route3'] = legs.route[legs.route != ''].iloc[2]
        except:pass
        df.loc[i, 'bus_capacity_upto_2hours'] = len(data['plan']['itineraries'])*30

        
    else:
        df.loc[i, 'transit_time'] =  np.nan
        df.loc[i, 'transfers'] = np.nan
        df.loc[i, 'walk_percent'] = np.nan
        df.loc[i, 'waiting_percent'] = np.nan
        # df.loc[i, 'trips'] = np.nan
        df.loc[i, 'trip1'] =  np.nan
        df.loc[i, 'trip2'] =  np.nan
        df.loc[i, 'trip3'] =  np.nan
        df.loc[i, 'route1'] =  np.nan
        df.loc[i, 'route2'] =  np.nan
        df.loc[i, 'route3'] =  np.nan
        df.loc[i, 'bus_capacity_upto_2hours'] = np.nan
    # print(df)    

In [8]:
def traveltimeDrive(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,CAR&date=05-04-2021&time="\
    +time+"&maxWalkDistance=750"
    r = requests.get(request)
    data=r.json()
    # print(latHome, lonHome, latWork, lonWork)
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [9]:
def traveltimeWalk(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK&date=05-04-2021&time="\
    +time+"&maxWalkDistance=50000"
    r = requests.get(request)
    data=r.json()
    # print(data['plan'])
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [10]:
def get_stop_url(geom):
    # print(df)
    # geom = df.geometry
    initial='http://127.0.0.1:5000/nearest/v1/driving/'
    end='?number=1'
    point = str(geom.x)+","+str(geom.y)
    requests2 = []
    requests2.append(initial+point+end)
    return requests2
    

In [11]:
import time
def nearby_points(reqs):
    url=reqs[0]
    #to remove any connection timeouts and port usage exceptions
    # r=None
    # l=None
    try:
        r=requests.get(url)
        data=r.json()
        l=Point(data['waypoints'][0]['location'])
        return l
    except:
        time.sleep(60)
        
    # return l

## merging tracts, cbgs name

In [12]:
tn_t = gpd.read_file('../Tennessee Census Tracts/tl_2016_47_tract.shp')
tn_bg = gpd.read_file('../Tennessee Census Block Groups/tl_2020_47_bg.shp')
tn_b = gpd.read_file('../Tennessee Census Block/tl_2020_47_tabblock20.shp')

In [13]:
ham_t = tn_t[tn_t['COUNTYFP'] == '065'].reset_index(drop=True)
ham_bg = tn_bg[tn_bg['COUNTYFP'] == '065'].reset_index(drop=True)
ham_b = tn_b[tn_b['COUNTYFP20'] == '065'].reset_index(drop=True)

In [14]:
ham_bg['name'] = ham_bg.GEOID.apply(lambda x: 'Block Group '+str(x[-1]) +', Census Tract '+str(x[6:9]).lstrip('0')+'.'+str(x[9:-1])+', Hamilton County, Tennessee' if str(x[9:-1]) != '00' else 'Block Group '+str(x[-1]) +', Census Tract '+str(x[6:9]).lstrip('0')+', Hamilton County, Tennessee')

In [15]:
ham_bg.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,name
0,47,065,011311,2,470650113112,Block Group 2,G5030,S,5199425,0,+35.0569904,-085.0889986,"POLYGON ((-85.10306 35.04920, -85.10302 35.049...","Block Group 2, Census Tract 113.11, Hamilton C..."
1,47,065,012300,3,470650123003,Block Group 3,G5030,S,10345344,828016,+35.0768325,-085.2581211,"POLYGON ((-85.28878 35.05388, -85.28877 35.054...","Block Group 3, Census Tract 123, Hamilton Coun..."
2,47,065,001200,2,470650012002,Block Group 2,G5030,S,882616,0,+35.0348647,-085.2633430,"POLYGON ((-85.27176 35.03502, -85.27162 35.035...","Block Group 2, Census Tract 12, Hamilton Count..."
3,47,065,001200,3,470650012003,Block Group 3,G5030,S,258744,0,+35.0413812,-085.2619178,"POLYGON ((-85.26688 35.03937, -85.26633 35.040...","Block Group 3, Census Tract 12, Hamilton Count..."
4,47,065,010303,2,470650103032,Block Group 2,G5030,S,2259086,0,+35.2116768,-085.1669565,"POLYGON ((-85.17869 35.21511, -85.17869 35.215...","Block Group 2, Census Tract 103.03, Hamilton C..."


In [16]:
ham_bg.to_csv('ham_cbg_withnames.csv', index=False)

## using cbgs

In [12]:
tn_cbg = gpd.read_file('../Tennessee Census Block Groups/tl_2020_47_bg.shp')
ham_cbg = tn_cbg[tn_cbg['COUNTYFP']=='065'].reset_index(drop=True)
ham_cbg.GEOID = ham_cbg.GEOID.astype(str)

In [13]:
cbgs = list(ham_cbg.GEOID.unique())

In [14]:
import itertools
combs = list(itertools.combinations(cbgs, 2))

In [55]:
ham_cbg['rep_pt'] = ham_cbg.geometry.apply(lambda x : x.representative_point())
ham_cbg['rep_pt_url'] = ham_cbg.rep_pt.apply(get_stop_url)
ham_cbg['rep_pt_nearby'] = ham_cbg.rep_pt_url.apply(nearby_points)

In [ ]:
ham_cbg['rep_pt_nearby_x'] = ham_cbg.rep_pt_nearby.apply(lambda pt : pt.x)
ham_cbg['rep_pt_nearby_y'] = ham_cbg.rep_pt_nearby.apply(lambda pt : pt.y)

In [179]:
ham_cbg_out = gpd.GeoDataFrame()
cnt= 0 
for i in combs:
    home = ham_cbg[ham_cbg.GEOID == i[0]]
    work = ham_cbg[ham_cbg.GEOID == i[1]]
    # print(home)
    ham_cbg_out.loc[cnt, 'home'] = home.GEOID.iloc[0]
    ham_cbg_out.loc[cnt, 'home_rep_pt_n'] = str(home.rep_pt_nearby.iloc[0])
    ham_cbg_out.loc[cnt, 'work'] = work.GEOID.iloc[0]
    ham_cbg_out.loc[cnt, 'work_rep_pt_n'] = str(work.rep_pt_nearby.iloc[0])
    traveltimeTransit2(ham_cbg_out, cnt, home.rep_pt_nearby_y.iloc[0], home.rep_pt_nearby_x.iloc[0], work.rep_pt_nearby_y.iloc[0], work.rep_pt_nearby_x.iloc[0], '08:00am')
    ham_cbg_out.loc[cnt, 'drive_time_fn'] = traveltimeDrive(home.rep_pt_nearby_y.iloc[0], home.rep_pt_nearby_x.iloc[0], work.rep_pt_nearby_y.iloc[0], work.rep_pt_nearby_x.iloc[0], '08:00am')
    ham_cbg_out.loc[cnt, 'walk_time_fn'] = traveltimeWalk(home.rep_pt_nearby_y.iloc[0], home.rep_pt_nearby_x.iloc[0], work.rep_pt_nearby_y.iloc[0], work.rep_pt_nearby_x.iloc[0], '08:00am')
    # print(ham_cbg_out)
    ham_cbg_out.loc[cnt, 'transit_score'] = np.divide(ham_cbg_out.loc[cnt, 'drive_time_fn'], ham_cbg_out.loc[cnt, 'transit_time'])
    cnt+=1
    

In [187]:
ham_cbg_out_5000 = ham_cbg_out

In [205]:
ham_cbg_out.to_csv('ham_cbg_allcombs_ts_5000.csv')

In [202]:
ham_cbg_out_5000.merge(ham_cbg[['GEOID', 'geometry']], left_on='home', right_on='GEOID', how='left').drop('GEOID', axis=1).rename({'geometry':'geometry_home'}, axis=1).merge(ham_cbg[['GEOID', 'geometry']], left_on='work', right_on='GEOID', how='left').drop('GEOID', axis=1).rename({'geometry':'geometry_work'}, axis=1).to_csv('ham_cbg_allcombs_ts_5000_geom.csv')

In [188]:
ham_cbg_out_750 = gpd.GeoDataFrame()
cnt= 0 
for i in combs:
    home = ham_cbg[ham_cbg.GEOID == i[0]]
    work = ham_cbg[ham_cbg.GEOID == i[1]]
    # print(home)
    ham_cbg_out_750.loc[cnt, 'home'] = home.GEOID.iloc[0]
    ham_cbg_out_750.loc[cnt, 'home_rep_pt_n'] = str(home.rep_pt_nearby.iloc[0])
    ham_cbg_out_750.loc[cnt, 'work'] = work.GEOID.iloc[0]
    ham_cbg_out_750.loc[cnt, 'work_rep_pt_n'] = str(work.rep_pt_nearby.iloc[0])
    traveltimeTransit2(ham_cbg_out_750, cnt, home.rep_pt_nearby_y.iloc[0], home.rep_pt_nearby_x.iloc[0], work.rep_pt_nearby_y.iloc[0], work.rep_pt_nearby_x.iloc[0], '08:00am')
    ham_cbg_out_750.loc[cnt, 'drive_time_fn'] = traveltimeDrive(home.rep_pt_nearby_y.iloc[0], home.rep_pt_nearby_x.iloc[0], work.rep_pt_nearby_y.iloc[0], work.rep_pt_nearby_x.iloc[0], '08:00am')
    ham_cbg_out_750.loc[cnt, 'walk_time_fn'] = traveltimeWalk(home.rep_pt_nearby_y.iloc[0], home.rep_pt_nearby_x.iloc[0], work.rep_pt_nearby_y.iloc[0], work.rep_pt_nearby_x.iloc[0], '08:00am')
    # print(ham_cbg_out_750)
    ham_cbg_out_750.loc[cnt, 'transit_score'] = np.divide(ham_cbg_out_750.loc[cnt, 'drive_time_fn'], ham_cbg_out_750.loc[cnt, 'transit_time'])
    cnt+=1
    

In [206]:
ham_cbg_out_750.merge(ham_cbg[['GEOID', 'geometry']], left_on='home', right_on='GEOID', how='left').drop('GEOID', axis=1).rename({'geometry':'geometry_home'}, axis=1).merge(ham_cbg[['GEOID', 'geometry']], left_on='work', right_on='GEOID', how='left').drop('GEOID', axis=1).rename({'geometry':'geometry_work'}, axis=1).to_csv('ham_cbg_allcombs_ts_750_geom.csv')

In [ ]:
# for i in range(hw_tracts_geom.shape[0]):
#     transit = traveltimeTransit(hw_tracts_geom, i, hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
#     hw_tracts_geom.loc[i, 'transit_time_fn'] = transit[0]
#     hw_tracts_geom.loc[i, 'legs'] = transit[1]
#     drive = traveltimeDrive(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
#     walk = traveltimeWalk(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
#     hw_tracts_geom.loc[i, 'transit_score'] = np.divide(hw_tracts_geom.loc[i, 'drive_time_fn'], hw_tracts_geom.loc[i, 'transit_time_fn'])

In [ ]:
hw_tracts_geom.groupby('TRACTCE_home').mean().reset_index()[['TRACTCE_home', 'transit_score']].to_csv('tract_transit_score_home.csv', index=False)
hw_tracts_geom.groupby('TRACTCE_work').mean().reset_index()[['TRACTCE_work', 'transit_score']].to_csv('tract_transit_score_work.csv', index=False)

In [26]:
ham_cbg_out_5000 = pd.read_csv('./ham_cbg_allcombs_ts_5000.csv', index_col=0)
ham_cbg_out_750 = pd.read_csv('./ham_cbg_allcombs_ts_750.csv', index_col=0)

In [27]:
ham_cbg_out_5000.home = ham_cbg_out_5000.home.astype(str)
ham_cbg_out_750.home = ham_cbg_out_750.home.astype(str)

In [28]:
ham_cbg_out_5000.groupby('home').mean().reset_index().merge(ham_bg[['GEOID', 'name']], left_on='home', right_on='GEOID').drop('GEOID', axis=1).to_csv('ham_cbg_allmoves_5000.csv', index=False)
ham_cbg_out_750.groupby('home').mean().reset_index().merge(ham_bg[['GEOID', 'name']], left_on='home', right_on='GEOID').drop('GEOID', axis=1).to_csv('ham_cbg_allmoves_750.csv', index=False)

In [38]:
ham_cbg_out_5000.groupby('home').mean().reset_index().merge(ham_bg[['GEOID', 'name', 'geometry']], left_on='home', right_on='GEOID').drop(['GEOID', 'work'], axis=1).to_csv('ham_cbg_allmoves_5000_geom.csv', index=False)
ham_cbg_out_750.groupby('home').mean().reset_index().merge(ham_bg[['GEOID', 'name', 'geometry']], left_on='home', right_on='GEOID').drop(['GEOID', 'work'], axis=1).to_csv('ham_cbg_allmoves_750_geom.csv', index=False)

In [24]:
lodes5000 = pd.read_csv('ham_lodes_cbg_ts_5000.csv')
lodes5000.h_geocode = lodes5000.h_geocode.astype(str)
lodes750 = pd.read_csv('ham_lodes_cbg_ts_750.csv')
lodes750.h_geocode = lodes750.h_geocode.astype(str)

In [34]:
lodes5000w = pd.read_csv('ham_lodes_cbg_ts_w_5000.csv')
lodes5000w.w_geocode = lodes5000w.w_geocode.astype(str)
lodes750w = pd.read_csv('ham_lodes_cbg_ts_w_750.csv')
lodes750w.w_geocode = lodes750w.w_geocode.astype(str)

In [29]:
lodes5000.groupby('h_geocode').mean().reset_index().merge(ham_bg[['GEOID', 'name']], left_on='h_geocode', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_lodes_5000.csv', index=False)
lodes750.groupby('h_geocode').mean().reset_index().merge(ham_bg[['GEOID', 'name']], left_on='h_geocode', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_lodes_750.csv', index=False)

In [28]:
lodes5000.groupby('h_geocode').mean().reset_index().merge(ham_bg[['GEOID', 'name', 'geometry']], left_on='h_geocode', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_lodes_5000_geom.csv', index=False)
lodes750.groupby('h_geocode').mean().reset_index().merge(ham_bg[['GEOID', 'name', 'geometry']], left_on='h_geocode', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_lodes_750_geom.csv', index=False)

In [32]:
grocery = pd.read_csv('./grocery_ts_750.csv')
grocery.home_cbg = grocery.home_cbg.astype(str)
grocery.groupby('home_cbg').mean().reset_index().merge(ham_bg[['GEOID', 'name']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_grocery_750.csv', index=False)

grocery2 = pd.read_csv('./grocery_ts_5000.csv')
grocery2.home_cbg = grocery2.home_cbg.astype(str)
grocery2.groupby('home_cbg').mean().reset_index().merge(ham_bg[['GEOID', 'name']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_grocery_5000.csv', index=False)

In [33]:
grocery.groupby('home_cbg').mean().reset_index().merge(ham_bg[['GEOID', 'name', 'geometry']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_grocery_750_geom.csv', index=False)
grocery2.groupby('home_cbg').mean().reset_index().merge(ham_bg[['GEOID', 'name', 'geometry']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1).dropna(subset=['walk_time_fn']).to_csv('ham_cbg_grocery_5000_geom.csv', index=False)

## cbg data info

In [40]:
ham_temp = pd.read_csv('./ham_cbg_allcombs_ts.csv')
ham_temp.drop(['home_rep_pt_n', 'work_rep_pt_n'], axis=1).dropna(subset=['transit_score']).to_csv('ham_temp.csv', index=False)

## using safegraph data

In [207]:
# safe_data = gpd.read_file('../Downloads/hamilton_allweeks2021_cbg_names.csv')

In [227]:
sg = gpd.read_file('../Downloads/ham_sg_week2_jan21.csv')
sg = sg.drop(['geometry', 'field_1'], axis=1)

In [265]:
import math
pd.set_option('display.max_columns', None)
sg.poi_cbg = sg.poi_cbg.astype(float).apply(lambda x: math.trunc(x)).astype(str)
sg_ham = sg.merge(ham_cbg['GEOID'], left_on=['home_cbg'], right_on='GEOID', how='inner').merge(ham_cbg['GEOID'], left_on=['poi_cbg'], right_on='GEOID', how='inner')

In [266]:
sg_ham

,date_begin,category_tags,naics_code,distance_from_home,includes_parking_lot,location_name,open_hours,raw_visit_counts,related_same_day_brand,related_same_week_brand,sub_category,top_category,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,home_cbg,poi_cbg,frequency,GEOID


In [292]:
ham_cbg_out_750.groupby('home').mean().reset_index().dropna(subset=['transit_score'])
ham_cbg_out_5000.groupby('home').mean().reset_index().dropna(subset=['transit_score'])

,home,transit_time,transfers,walk_percent,waiting_percent,trip3,bus_capacity_upto_2hours,drive_time_fn,walk_time_fn,transit_score
0,470650004001,3413.280702,0.631579,53.133020,4.469803,NaN,90.000000,1618.764706,12119.569767,0.352719
1,470650004002,3264.077586,0.491379,57.995004,3.962402,NaN,90.000000,1725.294737,12428.047120,0.395007
2,470650004003,3758.657343,0.552448,59.431439,5.226404,NaN,88.741259,1694.016194,12698.866935,0.321218
3,470650006001,3221.314286,0.400000,62.665125,1.785688,NaN,84.857143,1372.956522,9208.065217,0.352520
4,470650006002,3184.617647,0.382353,61.464474,2.053030,NaN,88.235294,1463.733333,9453.933333,0.390958
...,...,...,...,...,...,...,...,...,...,...
271,470650124003,3304.489796,0.040816,61.273376,0.078784,NaN,90.000000,1494.700000,10837.857143,0.368899
272,470650124004,3214.097222,0.027778,62.312557,0.383461,NaN,89.375000,1679.431452,12756.301205,0.407747
273,470659801001,5887.814815,0.648148,58.938592,4.879995,NaN,90.000000,1995.851852,16776.390244,0.296137
275,470659802002,7568.853846,1.092308,56.931134,3.996386,NaN,90.000000,2145.191111,19890.053097,0.253850


In [268]:
#grocery naics
import math
pd.set_option('display.max_columns', None)
grocery = sg[sg.naics_code == '445110.0']
grocery.poi_cbg = grocery.poi_cbg.astype(float).apply(lambda x: math.trunc(x)).astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,date_begin,category_tags,naics_code,distance_from_home,includes_parking_lot,location_name,open_hours,raw_visit_counts,related_same_day_brand,related_same_week_brand,sub_category,top_category,visits_by_day,visits_friday,visits_monday,visits_saturday,visits_sunday,visits_thursday,visits_tuesday,visits_wednesday,home_cbg,poi_cbg,frequency,home,work,drive_time_fn,walk_time_fn,transit_time,walk_percent,waiting_percent,transit_score
2,2021-01-04,Gourmet,445110.0,7660.0,false,Linda's Fruit Stand,nan,31.0,"{""Walmart"":8,""Food City"":8,""Dollar General"":8,...","{""Walmart"":29,""Mapco"":19,""McDonald's"":19,""Soni...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[0,7,6,8,6,3,1]",6,0,3,1,8,7,6,470650004002,470650020002,4.0,470650004002,470650020002,804.0,3660.0,1739.0,33.812536,0.115009,0.462335
4,2021-01-04,Gourmet,445110.0,8187.0,false,Linda's Produce,"{ ""Mon"": [[""8:00"", ""18:00""]], ""Tue"": [[""8:00"",...",23.0,"{""Walmart"":17,""Popeyes Louisiana Kitchen"":9,""D...","{""Walmart"":25,""Walgreens"":20,""Taco Bell"":15,""S...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[2,0,1,4,7,4,5]",7,2,4,5,4,0,1,470650101043,470650114135,5.0,470650101043,470650114135,3032.0,25856.0,NaN,NaN,NaN,NaN
8,2021-01-04,"Gourmet,Imported Food,Meats",445110.0,16895.0,false,Carniceria Loa No 7,nan,53.0,"{""Target"":4,""RaceWay"":4,""QuikTrip"":4,""Circle K...","{""Walmart"":36,""Sam's Club"":18,""QuikTrip"":15,""C...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[7,5,7,15,10,8,1]",10,7,8,1,15,5,7,470650033006,470650020001,4.0,470650033006,470650020001,846.0,5665.0,NaN,NaN,NaN,NaN
9,2021-01-04,"Gourmet,Imported Food,Meats",445110.0,16895.0,false,Carniceria Loa No 7,nan,53.0,"{""Target"":4,""RaceWay"":4,""QuikTrip"":4,""Circle K...","{""Walmart"":36,""Sam's Club"":18,""QuikTrip"":15,""C...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[7,5,7,15,10,8,1]",10,7,8,1,15,5,7,470650104313,470650020001,4.0,470650104313,470650020001,1786.0,12208.0,NaN,NaN,NaN,NaN
11,2021-01-04,International Grocery,445110.0,12671.0,false,Desi Brothers,"{ ""Mon"": [[""9:00"", ""21:00""]], ""Tue"": [[""9:00"",...",17.0,"{""Walmart"":24,""Sam's Club"":24,""Jack's Family R...","{""Walmart"":40,""Sam's Club"":33,""Jack's Family R...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[1,2,1,3,2,2,6]",2,1,2,6,3,2,1,470650104111,470650114453,4.0,470650104111,470650114453,2401.0,27726.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,2021-03-15,nan,445110.0,8895.0,false,Carniceria Loa,nan,85.0,"{""Walmart"":21,""Circle K Gas"":6,""Dollar General...","{""Walmart"":51,""McDonald's"":38,""Chick-fil-A"":21...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[17,5,7,13,8,18,17]",8,17,18,17,13,5,7,470650114134,470650034001,4.0,470650114134,470650034001,1679.0,12041.0,NaN,NaN,NaN,NaN
3352,2021-03-15,nan,445110.0,8895.0,false,Carniceria Loa,nan,85.0,"{""Walmart"":21,""Circle K Gas"":6,""Dollar General...","{""Walmart"":51,""McDonald's"":38,""Chick-fil-A"":21...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[17,5,7,13,8,18,17]",8,17,18,17,13,5,7,470650123001,470650034001,4.0,470650123001,470650034001,1158.0,7995.0,2683.0,19.083116,6.969810,0.431606
3354,2021-03-15,nan,445110.0,9092.0,false,Food City,"{ ""Mon"": [[""7:00"", ""22:00""]], ""Tue"": [[""7:00"",...",26.0,"{""Walmart"":15,""Family Dollar Stores"":12,""Circl...","{""Walmart"":35,""Circle K Gas"":30,""Chick-fil-A"":...",Supermarkets and Other Grocery (except Conveni...,Grocery Stores,"[2,4,6,3,4,5,2]",4,2,5,2,3,4,6,470650028003,470650114452,4.0,470650028003,470650114452,1182.0,8870.0,NaN,NaN,NaN,NaN
3355,2021-03-15,nan,445110.0,9092.0,false,Food City,"{ ""Mon"": [[""7:00"", ""22:00""]], ""Tue"": [[""7:00"",...",26.0,"{""Walmart"":15,""Family Dollar Stores"":12,""Circl...","{""Walmart"":35,""Circle K G

In [293]:
g1 = grocery.merge(ham_cbg_out_750[['home', 'work', 'drive_time_fn', 'walk_time_fn', 'transit_time', 'walk_percent', 'waiting_percent', 'transit_score']], how='left', left_on=['home_cbg', 'poi_cbg'], right_on=['home', 'work']).dropna(subset=['home'])
g1.groupby(['home_cbg', 'poi_cbg']).mean().reset_index()
g1.groupby(['home_cbg']).mean().reset_index().dropna(subset=['transit_score']).reset_index(drop=True).merge(ham_cbg[['GEOID', 'geometry']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1)
# .to_csv('grocery_ts_750.csv', index=False)

,home_cbg,drive_time_fn,walk_time_fn,transit_time,walk_percent,waiting_percent,transit_score,geometry
0,470650004001,975.400000,5058.300000,3746.700000,21.282782,15.765257,0.277944,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
1,470650004002,958.545455,5693.818182,1914.222222,34.816100,5.935628,0.364882,"POLYGON ((-85.29611 35.05021, -85.29596 35.050..."
2,470650006002,1223.000000,6960.666667,1834.000000,50.981461,0.109051,0.501091,"POLYGON ((-85.30483 35.06175, -85.30473 35.061..."
3,470650006003,309.000000,1346.000000,1849.000000,48.458626,8.653326,0.167117,"POLYGON ((-85.30922 35.05882, -85.30905 35.059..."
4,470650007001,428.000000,2021.000000,4232.000000,27.504726,3.780718,0.101134,"POLYGON ((-85.29225 35.08071, -85.29218 35.081..."
5,470650007002,324.000000,1726.000000,2189.000000,41.160347,7.309274,0.148013,"POLYGON ((-85.29443 35.07910, -85.29442 35.079..."
6,470650008001,755.277778,4935.777778,1255.923077,57.271289,1.201027,0.269189,"POLYGON ((-85.31936 35.07141, -85.31929 35.071..."
7,470650012001,1382.000000,13702.000000,5243.000000,7.018882,3.909975,0.263590,"POLYGON ((-85.26499 35.03854, -85.26478 35.038..."
8,470650013001,620.583333,3522.416667,2222.000000,51.363285,6.311413,0.267681,"POLYGON ((-85.27708 35.02520, -85.27693 35.025..."
9,470650013002,1071.000000,5703.000000,4827.000000,21.441889,26.206754,0.221877,"POLYGON ((-85.27430 35.02419, -85.27402 35.024..."


In [286]:
g1 = grocery.merge(ham_cbg_out_5000[['home', 'work', 'drive_time_fn', 'walk_time_fn', 'transit_time', 'walk_percent', 'waiting_percent', 'transit_score']], how='left', left_on=['home_cbg', 'poi_cbg'], right_on=['home', 'work']).dropna(subset=['home'])
g1.groupby(['home_cbg', 'poi_cbg']).mean().reset_index()
g1.groupby(['home_cbg']).mean().reset_index().reset_index(drop=True).merge(ham_cbg[['GEOID', 'geometry']], left_on='home_cbg', right_on='GEOID').drop('GEOID', axis=1).to_csv('grocery_ts_5000.csv', index=False)

### testing

In [116]:
i=46
temp_df = hw_tracts_geom
# temp_df.iloc[0]
# traveltimeTransit2(temp_df, i, temp_df.iloc[i].hc_nearby_y, temp_df.iloc[i].hc_nearby_x, temp_df.iloc[i].wc_nearby_y, temp_df.iloc[i].wc_nearby_x, '08:00am')
# traveltimeDrive( temp_df.iloc[i].hc_nearby_y, temp_df.iloc[i].hc_nearby_x, temp_df.iloc[i].wc_nearby_y, temp_df.iloc[i].wc_nearby_x, '08:00am')
traveltimeWalk( '36.301583', '-86.627562', '36.017136', '-85.598007', '08:00am')
# traveltimeWalk( '35.019977', '-85.141745', '35.049159', '-85.277', '08:00am')
# print(temp_df.iloc[i])

nan

## count bus stops in tracts

In [1]:
import gtfs_functions as gtfs
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('./2021-10-24.zip', busiest_date=False)

In [17]:
ham_bg

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,name
0,47,065,011311,2,470650113112,Block Group 2,G5030,S,5199425,0,+35.0569904,-085.0889986,"POLYGON ((-85.10306 35.04920, -85.10302 35.049...","Block Group 2, Census Tract 113.11, Hamilton C..."
1,47,065,012300,3,470650123003,Block Group 3,G5030,S,10345344,828016,+35.0768325,-085.2581211,"POLYGON ((-85.28878 35.05388, -85.28877 35.054...","Block Group 3, Census Tract 123, Hamilton Coun..."
2,47,065,001200,2,470650012002,Block Group 2,G5030,S,882616,0,+35.0348647,-085.2633430,"POLYGON ((-85.27176 35.03502, -85.27162 35.035...","Block Group 2, Census Tract 12, Hamilton Count..."
3,47,065,001200,3,470650012003,Block Group 3,G5030,S,258744,0,+35.0413812,-085.2619178,"POLYGON ((-85.26688 35.03937, -85.26633 35.040...","Block Group 3, Census Tract 12, Hamilton Count..."
4,47,065,010303,2,470650103032,Block Group 2,G5030,S,2259086,0,+35.2116768,-085.1669565,"POLYGON ((-85.17869 35.21511, -85.17869 35.215...","Block Group 2, Census Tract 103.03, Hamilton C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,47,065,011314,3,470650113143,Block Group 3,G5030,S,2073308,0,+35.0418931,-085.0841881,"POLYGON ((-85.10007 35.04181, -85.10003 35.041...","Block Group 3, Census Tract 113.14, Hamilton C..."
274,47,065,011314,1,470650113141,Block Group 1,G5030,S,1407238,0,+35.0395608,-085.1053233,"POLYGON ((-85.11560 35.04317, -85.11470 35.043...","Block Group 1, Census Tract 113.14, Hamilton C..."
275,47,065,010901,1,470650109011,Block Group 1,G5030,S,2957834,0,+35.0691007,-085.3246938,"POLYGON ((-85.33303 35.05349, -85.33288 35.054...","Block Group 1, Census Tract 109.01, Hamilton C..."
276,47,065,980200,1,470659802001,Block Group 1,G5030,S,8635762,0,+35.0809406,-085.1393704,"POLYGON ((-85.16139 35.07340, -85.16136 35.073...","Block Group 1, Census Tract 802, Hamilton Coun..."


In [22]:
gpd.sjoin(stops[['stop_id', 'geometry']], ham_bg).groupby('GEOID').count().reset_index()[['GEOID', 'stop_id']].rename({'stop_id':'stops'}, axis=1).merge(ham_bg[['GEOID', 'geometry']], on='GEOID').to_csv('ham_cbg_busstops.csv', index=False)

In [90]:
hw_tracts_geom2 = hw_tracts_geom
hw_tracts_geom2['under_30_min_transit'] = hw_tracts_geom2.transit_time.apply(lambda x: 1 if x <= 1800 else 0)
hw_tracts_geom2['under_45_min_transit'] = hw_tracts_geom2.transit_time.apply(lambda x: 1 if x <= 2400 else 0)
hw_tracts_geom2['under_60_min_transit'] = hw_tracts_geom2.transit_time.apply(lambda x: 1 if x <= 3600 else 0)
# .groupby('TRACTCE20_home').mean().reset_index()[['TRACTCE20_home', 'transit_score']]

In [101]:
hw_tracts_geom2.groupby('TRACTCE20_home').sum().reset_index()[['TRACTCE20_home', 'under_30_min_transit', 'under_45_min_transit', 'under_60_min_transit']].to_csv('tracts_ts_time_levels.csv', index=False)